# Advanced ML: Домашнее задание 2

[сслыка](https://docs.google.com/document/d/1wOSw5ZX8cY9aL2wM2b3LzPadofxW0hFnz0t9HIkdgq4/edit#heading=h.2yf7hlrm983) на описание задания 

`Мысли по ходу решения задачи`

1) Мы должны будем получить модель, которая обучится по игроку и вопросу предсказывать: ответил человек или нет на вопрос
2) Из модели, т.к. это линрег мы получим веса для каждого вопроса и игрока. Теперь надо понять: если у вопроса большой вес, это значит что на него мало или много кто ответил? если у игрока большой вес, это значит, что он на всё подря отвечает или нет?
3) Получив веса игроков мы можем оценить силу нашей команды. Мы не знаем какие нам будут приходить вопросы, но знаем как наш игрок себя ведёт (есть вес в линреге), т.е. он раньше хорошо отвечал -> и теперь будет хорошо отвечать. Вес для команды можно оценить как либо среднее между всеми, либо максимум из её участников, либо сумма. Наиболее логичным было бы взять сумму, т.к. она учитывает силы всех членов команды и зависит от числа игроков. Больше игроков -> больше шанс правильно ответить
4) Нам надо будет сделать новые данные для EM, которые будут иметь вид какой-то такой: 1 для вопроса, на который отвечаем, 1 для всех игроков, которые были в одной команде и отвечали на него совместно, когда был задан вопрос, 1 (label или target) если команда ответила правильно

теперь попробую помыслить сюда по поводу весов у линрега для игрока и вопроса

если у меня большой вес у игрока, значит я скорее всего превышу трешхолд и дам ответ 1
т.е. игрок ответит на вопрос

делаю вывод: большой коэф у игрока говорит о том, что он сильный и отвечает в пределе на любые вопросы

если же у вопроса большой коэф, то не зависимо от игрока, сокрее всего я превышу порог и значит дам правильный ответ

вывод: если вес большой у вопроса, то вопрос простой

5) В ЕМ алгоритме нам известно, что игрок точно не ответил, если команда не ответила. Однако, если команда дала ответ - нельзя быть уверенным, что ответ дал именно тот или иной игрок. 

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle 
import typing as tp

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from scipy.stats import kendalltau, spearmanr

## Подготовка данных

In [2]:
with open('players.pkl', 'rb') as f:
    data = pickle.load(f)
players = pd.DataFrame.from_dict(data, orient='index')

with open('tournaments.pkl', 'rb') as f:
    data = pickle.load(f)
tournaments = pd.DataFrame.from_dict(data, orient='index')

with open('results.pkl', 'rb') as f:
    results = pickle.load(f)

In [3]:
players.sample(5)

,id,name,patronymic,surname
73083,73083,Ирина,,Гукасян
167421,167421,Василий,Васильевич,Подушкин
67702,67702,Олег,,Демьянов
18974,18974,Мария,,Лыкова
111101,111101,Никита,Сергеевич,Веселов


In [4]:
tournaments.sample(5)

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
4629,4629,Щит и меч,2017-11-26T10:00:00+03:00,2017-11-26T15:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/51,"[{'id': 4715, 'name': 'Сергей', 'patronymic': ...",None,"{'1': 15, '2': 15, '3': 15, '4': 15}"
574,574,Азовский бриз,2010-01-16T00:00:00+03:00,2010-01-17T00:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/9,[],None,None
4579,4579,Записки охотника,2017-10-21T10:00:00+03:00,2017-10-25T10:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/51,"[{'id': 16731, 'name': 'Владимир', 'patronymic...",{'dateRequestsAllowedTo': '2017-10-25T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5550,5550,Студенческая лига Казани,2018-09-29T00:00:00+03:00,2019-03-10T14:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/52,"[{'id': 46976, 'name': 'Мансур', 'patronymic':...",None,"{'1': 12, '2': 12, '3': 12, '4': 12, '5': 12, ..."
3929,3929,Серия Premier. Громокипящий кубок,2017-01-06T20:00:00+03:00,2017-01-10T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/50,"[{'id': 36502, 'name': 'Сергей', 'patronymic':...",{'dateRequestsAllowedTo': '2017-01-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"


In [5]:
results.keys(), results[1][0].keys()

(dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,

In [6]:
# У каких-то результатов отсутствует поле mask
results[6471][0].keys()

dict_keys(['team', 'current', 'synchRequest', 'flags', 'teamMembers'])

In [7]:
train_tournaments = tournaments[tournaments['dateStart'].str.contains('2019-')]
train_tournaments_ids = train_tournaments["id"].values.tolist()
train_tournaments_clear_ids = [
    tour_id for tour_id in train_tournaments_ids if 
    (len(results[tour_id]) > 1) and 
    ('mask' in results[tour_id][0].keys()) and 
    (
        all(
            results[tour_id][team_idx]['mask'] is not None 
            for team_idx in range(len(results[tour_id]))
        )
    )
]
train_tournaments = train_tournaments.loc[train_tournaments_clear_ids]

test_tournaments = tournaments[tournaments['dateStart'].str.contains('2020-')]
test_tournaments_ids = test_tournaments["id"].values.tolist()
test_tournaments_clear_ids = [
    tour_id for tour_id in test_tournaments_ids if 
    (len(results[tour_id]) > 1) and 
    ('mask' in results[tour_id][0].keys()) and 
    (
        all(
            results[tour_id][team_idx]['mask'] is not None 
            for team_idx in range(len(results[tour_id]))
        )
    )
]
test_tournaments = test_tournaments.loc[test_tournaments_clear_ids]

train_tournaments.shape, test_tournaments.shape

((669, 9), (168, 9))

In [8]:
train_tournaments

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
4772,4772,Синхрон северных стран. Зимний выпуск,2019-01-05T19:00:00+03:00,2019-01-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 28379, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-01-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4973,4973,Балтийский Берег. 3 игра,2019-01-25T19:05:00+03:00,2019-01-29T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-01-28T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4974,4974,Балтийский Берег. 4 игра,2019-03-01T19:05:00+03:00,2019-03-05T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-03-04T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4975,4975,Балтийский Берег. 5 игра,2019-04-05T19:05:00+03:00,2019-04-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-04-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
4986,4986,ОВСЧ. 6 этап,2019-02-15T20:00:00+03:00,2019-02-19T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 59140, 'name': 'Борис', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-19T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
...,...,...,...,...,...,...,...,...,...
6173,6173,Кубок Мэра Казани,2019-12-15T10:00:00+03:00,2019-12-15T15:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 33624, 'name': 'Ренат', 'patronymic': ...",None,"{'1': 15, '2': 15, '3': 15, '4': 15}"
6191,6191,Всеармянский Интеллектуальный Фестиваль,2019-12-22T12:00:00+03:00,2019-12-22T16:00:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 19981, 'name': 'Сейран', 'patronymic':...",None,"{'1': 12, '2': 12, '3': 12}"
6249,6249,Школьный синхрон-lite. Сезон 3,2019-09-01T00:05:00+03:00,2020-04-30T23:55:00+03:00,"{'id': 5, 'name': 'Общий зачёт'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",None,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ..."
6254,6254,Школьная лига,2019-10-04T19:00:00+03:00,2020-03-22T19:00:00+03:00,"{'id': 5, 'name': 'Общий зачёт'}",/seasons/53,"[{'id': 39218, 'name': 'Владислав', 'patronymi...",None,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ..."


In [9]:
for i, team in enumerate(results[train_tournaments["id"][4772]]):
    print(team["team"]["name"], team["mask"], team["position"])
    for member in team["teamMembers"]:
        print("ID участника команды:", member["player"]["id"])
    if i >= 2:
        break

Рабочее название 111111111011111110111111111100010010 1
ID участника команды: 6212
ID участника команды: 18332
ID участника команды: 18036
ID участника команды: 22799
ID участника команды: 15456
ID участника команды: 26089
Сборная Бутана 111111111011110100101111011001011010 5.5
ID участника команды: 1585
ID участника команды: 40840
ID участника команды: 1584
ID участника команды: 10998
ID участника команды: 16206
Ять 111111111011110101101111001011110000 5.5
ID участника команды: 23513
ID участника команды: 18168
ID участника команды: 21060
ID участника команды: 35850
ID участника команды: 31332
ID участника команды: 10187


## Baseline model

In [10]:
baseline_data = []
baseline_labels = ["member_id", "question_id", "target"]

for tournament_id in train_tournaments["id"]:
    for team in results[tournament_id]:
        for member in team["teamMembers"]:
            for i, answer in enumerate(team["mask"]):
                if answer in ["1", "0"]:
                    member_id = member["player"]["id"]
                    question_id = f"t{tournament_id}_q{i}"
                    baseline_data += [[member_id, question_id, answer]]

baseline_df = pd.DataFrame.from_records(
    data=baseline_data,
    columns=baseline_labels
)

In [11]:
baseline_df["target"] = baseline_df["target"].astype(int)
baseline_df["member_id"] = baseline_df["member_id"].astype("category") # Pandas memory usage optimization

In [12]:
baseline_train = pd.get_dummies(baseline_df[["member_id", "question_id"]], sparse = True)

In [13]:
baseline_train

,member_id_15,member_id_16,member_id_23,member_id_31,member_id_35,member_id_38,member_id_47,member_id_59,member_id_65,member_id_79,...,question_id_t6255_q90,question_id_t6255_q91,question_id_t6255_q92,question_id_t6255_q93,question_id_t6255_q94,question_id_t6255_q95,question_id_t6255_q96,question_id_t6255_q97,question_id_t6255_q98,question_id_t6255_q99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20864567,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20864568,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20864569,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20864570,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [94]:
baseline_model = LinearRegression()
baseline_model.fit(baseline_train, baseline_df["target"])

LinearRegression()

In [95]:
baseline_model.coef_

array([ 0.15873333,  0.3339047 ,  0.23405704, ...,  0.09254016,
        0.13424821, -0.15742442])

In [131]:
def get_players_scores(
    model: tp.Optional[LinearRegression | LogisticRegression],
    data: pd.DataFrame
) -> dict:
    player2score = {}
    for i, column in enumerate(data.columns):
        if "member_id" in column:
            member_id = int(column.split("_")[-1])
            if isinstance(model, LinearRegression):
                player2score[member_id] = model.coef_[i]
            elif isinstance(model, LogisticRegression):
                player2score[member_id] = model.coef_[0][i]
            else:
                raise ValueError("Model must be [LinearRegression | LogisticRegression]")
    return player2score

In [132]:
def get_questions_scores(
    model: tp.Optional[LinearRegression | LogisticRegression],
    data: pd.DataFrame
) -> dict:
    question2score = {}
    for i, column in enumerate(data.columns):
        if "question_id" in column:
            question_id = int(column.split("_")[-1][1:])
            if isinstance(model, LinearRegression):
                question2score[question_id] = model.coef_[i]
            elif isinstance(model, LogisticRegression):
                question2score[question_id] = model.coef_[0][i]
            else:
                raise ValueError("Model must be [LinearRegression | LogisticRegression]")
    return question2score

In [133]:
player2score = get_players_scores(baseline_model, baseline_train)
player2score

{15: 0.15873333332982575,
 16: 0.3339046976403095,
 23: 0.2340570427700855,
 31: 0.2008643901932596,
 35: 0.20129832232282674,
 38: 0.0203593158307938,
 47: 0.3643087809006876,
 59: 0.2245890769568209,
 65: 0.1025029870097813,
 79: 0.19002531754022436,
 80: 0.3737752972073255,
 82: 0.1466595788473423,
 98: 0.03105168323119793,
 112: 0.14017866947192417,
 113: 0.2111929783391495,
 117: 0.4057053328903156,
 119: 0.18291163170960448,
 133: 0.3522778959807179,
 136: 0.09878839677686861,
 144: 0.5003233177572097,
 150: 0.356915407819682,
 153: 0.3306179030445191,
 157: 0.10213151113394907,
 160: 0.4167018448213091,
 176: 0.43143869110174293,
 178: 0.22667908583277904,
 182: 0.24780266261427245,
 196: 0.18911850638786948,
 223: 0.40738099512231324,
 230: 0.33771102361748545,
 232: 0.11875309406497993,
 233: 0.11221680377489004,
 236: 0.4506568118157002,
 261: 0.22248007742210418,
 263: 0.18695407470752826,
 278: 0.31691226842365183,
 286: -0.014073540716111404,
 315: 0.383136126460008,
 322:

## Оценка качетсва рейтинг-системы

In [98]:
def calc_corrs(player2score: dict) -> tp.Tuple:
    spearmanr_corr = []
    kendalltau_corr = []

    for tournament_id in test_tournaments["id"]:
        real_position = {}
        predicted_position = {}
        
        for i, team in enumerate(results[tournament_id]):
            if len(team["teamMembers"]) == 0:
                continue
            
            team_sum_score = sum([
                player2score[member["player"]["id"]] 
                if member["player"]["id"] in player2score.keys()
                else 0
                for member in team["teamMembers"]
            ])
            real_position[team["team"]["id"]] = team["position"]
            predicted_position[team["team"]["id"]] = team_sum_score
        
        predicted_position = {
            k: i for i, (k, _) in enumerate(sorted(predicted_position.items(), key=lambda item: -item[1]))
        }
        
        predicted_position = {
            k: predicted_position[k] for k, _ in real_position.items()
        }
        
        kendalltau_corr += [kendalltau(list(real_position.values()), list(predicted_position.values())).statistic]
        spearmanr_corr += [spearmanr(list(real_position.values()), list(predicted_position.values())).statistic]
    
    return (np.mean(kendalltau_corr), np.mean(spearmanr_corr))

In [99]:
kendall, spearman = calc_corrs(player2score)
print("Средняя кореляция Кендалла ", kendall)
print("Средняя кореляция Спирмена ", spearman)

Средняя кореляция Кендалла  0.6271063689294365
Средняя кореляция Спирмена  0.7841440627459042


## EM алгоритм

In [85]:
em_data = []
em_labels = ["members_ids", "question_id", "target"]

for tournament_id in train_tournaments["id"]:
    for team in results[tournament_id]:
        team_members_ids = [
            "member_id_"+str(member["player"]["id"]) for member in team["teamMembers"]
        ]
        for i, answer in enumerate(team["mask"]):
            if answer in ["1", "0"]:
                question_id = f"question_id_t{tournament_id}_q{i}"
                em_data += [[team_members_ids, question_id, answer]]

em_df = pd.DataFrame.from_records(
    data=em_data,
    columns=em_labels
)

In [86]:
em_df

,members_ids,question_id,target
0,"[member_id_6212, member_id_18332, member_id_18...",t4772_q0,1
1,"[member_id_6212, member_id_18332, member_id_18...",t4772_q1,1
2,"[member_id_6212, member_id_18332, member_id_18...",t4772_q2,1
3,"[member_id_6212, member_id_18332, member_id_18...",t4772_q3,1
4,"[member_id_6212, member_id_18332, member_id_18...",t4772_q4,1
...,...,...,...
3798708,"[member_id_217153, member_id_217154, member_id...",t6255_q31,0
3798709,"[member_id_217153, member_id_217154, member_id...",t6255_q32,0
3798710,"[member_id_217153, member_id_217154, member_id...",t6255_q33,0
3798711,"[member_id_217153, member_id_217154, member_id...",t6255_q34,0


In [87]:
mlb = MultiLabelBinarizer(sparse_output=True)
em_train = em_df.join(
    pd.DataFrame.sparse.from_spmatrix(
        mlb.fit_transform(em_df["members_ids"]),
        index=em_df.index,
        columns=mlb.classes_
    )
).drop(["members_ids", "question_id", "target"], axis=1)
one_hot_questions = pd.get_dummies(em_df["question_id"], sparse = True)
em_train = em_train.join(one_hot_questions)

In [228]:
em_target = em_df["target"].to_numpy(dtype=int)
em_target

array([1, 1, 1, ..., 0, 0, 0])

In [123]:
em_train

,member_id_1000,member_id_100010,member_id_100013,member_id_100019,member_id_100026,member_id_100031,member_id_100039,member_id_100046,member_id_100061,member_id_100066,...,t6255_q90,t6255_q91,t6255_q92,t6255_q93,t6255_q94,t6255_q95,t6255_q96,t6255_q97,t6255_q98,t6255_q99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3798708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3798709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3798710,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3798711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

Этот код работает достаточно долго. 

~ 10 минут на 5 эпох

Пытаемся предсказать, что команда с данным составом даст правильный ответ. Считая такую статистику по всем командам, мы будем оценивать вклад каждого игрока в свою команду. 

Однако нам не надо предсказывать что-либо в случае, если вся команд дала целиком неверный ответ. В данном случае мы заменяем новый таргет нашими исходными данными.

In [238]:
epoch_num = 5
target = em_target

for epoch in range(1, epoch_num + 1):
    em_model = LinearRegression(n_jobs=-1)
    em_model.fit(em_train, target)
    
    target = sigmoid(em_model.predict(em_train)) 
    target[em_target == 0] = 0
    
    em_player2score = get_players_scores(em_model, em_train)
    kendall, spearman = calc_corrs(em_player2score)
    print(f"------------------ Epoch {epoch}/{epoch_num} ------------------")
    print("Средняя кореляция Кендалла ", kendall)
    print("Средняя кореляция Спирмена ", spearman)

------------------ Epoch 1/5 ------------------
Средняя кореляция Кендалла  0.5470909745144753
Средняя кореляция Спирмена  0.6993070686076981
------------------ Epoch 2/5 ------------------
Средняя кореляция Кендалла  0.5487164993072747
Средняя кореляция Спирмена  0.7019076029821836
------------------ Epoch 3/5 ------------------
Средняя кореляция Кендалла  0.5487861035843812
Средняя кореляция Спирмена  0.7020399489564245
------------------ Epoch 4/5 ------------------
Средняя кореляция Кендалла  0.5489784318298768
Средняя кореляция Спирмена  0.7021226245668215
------------------ Epoch 5/5 ------------------
Средняя кореляция Кендалла  0.5490762263942183
Средняя кореляция Спирмена  0.7021764297317749


## Рейтинг-лист турниров

In [242]:
question2scores = get_questions_scores(baseline_model, baseline_train)

Чем больше коэфициент у вопроса, тем больше вклада он даст в вероятность правильности ответа, вне зависимости от отвечающего. Значит наибольшие коэф отвечают за простые вопросы. 

In [239]:
tournaments_scores = {}

for tournament_id in train_tournaments["id"]:
    tournaments_scores[tournament_id] = np.mean([
        question2scores[int(question.split("_")[-1][1:])]
        for question in baseline_train.columns 
        if "question_id_t"+str(tournament_id) in question
    ])    
    
predicted_position = {
    k: i for i, (k, _) in enumerate(sorted(tournaments_scores.items(), key=lambda item: -item[1]))
}

Турниры с грузными названия ми по типу `Лига наций` оказались в начале, наверное это и есть серьезные турниры. Правда не появился ниодин чемпионат мира, и затесался какой-то Школьный синхрон, что немного странно. 

In [240]:
tournaments.loc[list(predicted_position.keys())[:10]]

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
5827,5827,Шестой киевский марафон. Асинхрон,2019-08-02T14:00:00+03:00,2019-11-24T14:00:00+03:00,"{'id': 8, 'name': 'Асинхрон'}",/seasons/53,"[{'id': 7420, 'name': 'Эдуард', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-11-23T23:59:59...,"{'1': 100, '2': 100, '3': 100, '4': 100, '5': ..."
6085,6085,Серия Гран-при. Общий зачёт,2019-09-06T19:00:00+03:00,2020-04-14T19:00:00+03:00,"{'id': 5, 'name': 'Общий зачёт'}",/seasons/53,"[{'id': 36502, 'name': 'Сергей', 'patronymic':...",None,"{'1': 45, '2': 45, '3': 45, '4': 45, '5': 45, ..."
6249,6249,Школьный синхрон-lite. Сезон 3,2019-09-01T00:05:00+03:00,2020-04-30T23:55:00+03:00,"{'id': 5, 'name': 'Общий зачёт'}",/seasons/53,"[{'id': 23740, 'name': 'Владимир', 'patronymic...",None,"{'1': 36, '2': 36, '3': 36, '4': 36, '5': 36, ..."
5060,5060,Шестая октава: СИ-Мажор,2019-02-22T20:00:00+03:00,2019-02-26T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 32099, 'name': 'Сергей', 'patronymic':...",{'dateRequestsAllowedTo': '2019-02-26T23:59:59...,"{'1': 13, '2': 13, '3': 13}"
5434,5434,Шестая октава: СИ-Мажор. Лига Наций: Украина,2019-02-22T20:00:00+03:00,2019-02-26T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 32099, 'name': 'Сергей', 'patronymic':...",{'dateRequestsAllowedTo': '2019-02-26T23:59:59...,"{'1': 13, '2': 13, '3': 13}"
5435,5435,Шестая октава: СИ-Мажор. Лига Наций: Прибалтика,2019-02-22T20:00:00+03:00,2019-02-26T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 32099, 'name': 'Сергей', 'patronymic':...",{'dateRequestsAllowedTo': '2019-02-26T23:59:59...,"{'1': 13, '2': 13, '3': 13}"
5436,5436,Шестая октава: СИ-Мажор. Лига Наций: Беларусь,2019-02-22T20:00:00+03:00,2019-02-26T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 40393, 'name': 'Юрий', 'patronymic': '...",{'dateRequestsAllowedTo': '2019-02-26T23:59:59...,"{'1': 13, '2': 13, '3': 13}"
5141,5141,Азовский бриз,2019-01-18T19:00:00+03:00,2019-01-22T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 15755, 'name': 'Александр', 'patronymi...",{'dateRequestsAllowedTo': '2019-01-22T23:59:59...,"{'1': 12, '2': 12, '3': 12, '4': 12}"
5061,5061,Октавы: Гала-турнир,2019-05-11T15:00:00+03:00,2019-05-15T15:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 32099, 'name': 'Сергей', 'patronymic':...",{'dateRequestsAllowedTo': '2019-05-15T23:59:59...,"{'1': 13, '2': 13, '3': 13}"
5757,5757,Открытый Кубок России,2019-12-05T19:00:00+03:00,2019-12-11T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 23030, 'name': 'Марина', 'patronymic':...",{'dateRequestsAllowedTo': '2019-12-08T23:55:00...,"{'1': 12, '2': 12, '3': 12, '4': 12}"


In [241]:
tournaments.loc[list(predicted_position.keys())[-10:]]

,id,name,dateStart,dateEnd,type,season,orgcommittee,synchData,questionQty
5431,5431,Rock Around the Clock,2019-02-02T08:00:00+03:00,2019-02-06T08:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 137496, 'name': 'Руслан', 'patronymic'...",{'dateRequestsAllowedTo': '2019-02-05T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5420,5420,Львов зимой. Барон,2019-01-26T12:00:00+03:00,2019-01-30T12:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 40393, 'name': 'Юрий', 'patronymic': '...",{'dateRequestsAllowedTo': '2019-01-29T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5629,5629,Синхрон простых вопросов. Лето,2019-07-19T19:00:00+03:00,2019-07-23T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 36502, 'name': 'Сергей', 'patronymic':...",{'dateRequestsAllowedTo': '2019-07-23T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5154,5154,Синхрон Студенческого Кубка Княгини Ольги,2019-04-05T19:00:00+03:00,2019-04-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 40393, 'name': 'Юрий', 'patronymic': '...",{'dateRequestsAllowedTo': '2019-04-09T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5890,5890,Осенний синхронный Умлаут,2019-11-14T08:00:00+03:00,2019-11-20T23:55:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 21375, 'name': 'Максим', 'patronymic':...",{'dateRequestsAllowedTo': '2019-11-20T14:00:00...,"{'1': 12, '2': 12, '3': 12}"
5952,5952,Зеркало Симбирск-Open,2019-09-27T18:00:00+03:00,2019-10-01T18:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 56850, 'name': 'Артём', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-10-01T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5895,5895,Взгляд отца,2019-12-19T20:00:00+03:00,2019-12-26T20:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 19468, 'name': 'Виктория', 'patronymic...",{'dateRequestsAllowedTo': '2019-12-26T19:00:00...,"{'1': 12, '2': 12, '3': 12}"
5756,5756,Жизнь и время Михаэля К.,2019-10-03T19:00:00+03:00,2019-10-09T19:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/53,"[{'id': 14544, 'name': 'Константин', 'patronym...",{'dateRequestsAllowedTo': '2019-10-08T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
5429,5429,Розовая лента,2019-02-08T23:00:00+03:00,2019-02-12T23:00:00+03:00,"{'id': 3, 'name': 'Синхрон'}",/seasons/52,"[{'id': 33331, 'name': 'Дарья', 'patronymic': ...",{'dateRequestsAllowedTo': '2019-02-11T23:59:59...,"{'1': 12, '2': 12, '3': 12}"
6090,6090,Дзержинский марафон,2019-12-21T15:30:00+03:00,2019-12-22T15:30:00+03:00,"{'id': 2, 'name': 'Обычный'}",/seasons/53,"[{'id': 7551, 'name': 'Алексей', 'patronymic':...",None,"{'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, ..."
